In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# === Parametri ===
ticker_symbol = "^SPX"      # S&P 500
target_expiration = "2026-03-20"  # Scadenza opzioni
target_strike = 4000        # Strike desiderato (da regolare in base al livello attuale dell'SPX)
days_to_display = 15       # Numero di giorni da visualizzare sul grafico

# === Scarica storico prezzi del sottostante ===
end_date = datetime.today()
start_date = end_date - timedelta(days=days_to_display)
underlying = yf.download(ticker_symbol, start=start_date, end=end_date, interval="1d", auto_adjust=False)

# === Scarica catena opzioni ===
ticker = yf.Ticker(ticker_symbol)
opt_chain = ticker.option_chain(target_expiration)
calls = opt_chain.calls
puts = opt_chain.puts

# === Trova l'opzione Call e Put allo strike desiderato ===
try:
    call_symbol = calls.loc[calls['strike'] == target_strike, 'contractSymbol'].values[0]
    put_symbol = puts.loc[puts['strike'] == target_strike, 'contractSymbol'].values[0]
except IndexError:
    raise ValueError(f"Nessuna opzione trovata per strike {target_strike}. Strike disponibili: {calls['strike'].tolist()}")

# === Scarica storico prezzi opzioni ===
call_data = yf.download(call_symbol, start=start_date, end=end_date, interval="1d", auto_adjust=False)
put_data = yf.download(put_symbol, start=start_date, end=end_date, interval="1d", auto_adjust=False)

# === Calcola la crescita percentuale ===
call_data['Close_pct_change'] = call_data['Close'].pct_change().fillna(0).add(1).cumprod().sub(1).mul(100)
put_data['Close_pct_change'] = put_data['Close'].pct_change().fillna(0).add(1).cumprod().sub(1).mul(100)
underlying['Close_pct_change'] = underlying['Close'].pct_change().fillna(0).add(1).cumprod().sub(1).mul(100)


# === Grafico ===
fig, ax1 = plt.subplots(figsize=(12,6))

# Prezzi Call e Put in percentuale
ax1.plot(call_data.index, call_data['Close_pct_change'], label=f"Call {target_strike}", color='blue')
ax1.plot(put_data.index, put_data['Close_pct_change'], label=f"Put {target_strike}", color='red')
ax1.set_xlabel("Data")
ax1.set_ylabel("Crescita Percentuale (%)")
ax1.legend(loc="upper left")

# Add value labels to the put line using the actual 'Close' values
for i in range(len(put_data)):
    ax1.annotate(f'{put_data["Close"].iloc[i].iloc[0]:.2f}', (put_data.index[i], put_data['Close_pct_change'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')


# Prezzo sottostante su secondo asse in percentuale
ax2 = ax1.twinx()
ax2.plot(underlying.index, underlying['Close_pct_change'], label="Sottostante", color='green', linestyle="--")
ax2.set_ylabel("Crescita Percentuale Sottostante (%)")


plt.title(f"Andamento Percentuale Call & Put Strike {target_strike} - Scadenza {target_expiration} - {ticker_symbol}")
plt.show()

In [ ]:
# === Scarica le date di scadenza disponibili ===
expirations = ticker.options

# === Filtra le scadenze per includere solo il terzo venerdì del mese ===
from pandas.tseries.offsets import BDay
filtered_expirations = []
for exp in expirations:
    try:
        exp_date = datetime.strptime(exp, '%Y-%m-%d')
        # Check if it's the third Friday of the month
        # Find the first day of the month
        first_day_of_month = exp_date.replace(day=1)
        # Find the first Friday of the month
        first_friday = first_day_of_month + BDay(0) # Start with the first day
        while first_friday.weekday() != 4: # 4 is Friday
            first_friday += BDay(1)
        # Calculate the third Friday
        third_friday = first_friday + BDay(10) # 2 weeks after the first Friday (10 business days)

        if exp_date.date() == third_friday.date():
             filtered_expirations.append(exp)
    except ValueError:
        # Handle potential errors in date parsing
        continue


# === Scarica dati per le Put su diverse scadenze filtrate e raccogli prezzo e implied volatility ===
put_data_summary = {}
for exp in filtered_expirations:
    try:
        opt_chain_exp = ticker.option_chain(exp)
        puts_exp = opt_chain_exp.puts
        # Find the put option for the target strike
        put_option = puts_exp.loc[puts_exp['strike'] == target_strike]
        if not put_option.empty:
            # Get the last close price and implied volatility
            last_price = put_option['lastPrice'].values[0]
            implied_volatility = put_option['impliedVolatility'].values[0]
            put_data_summary[exp] = {'lastPrice': last_price, 'impliedVolatility': implied_volatility}
    except (IndexError, Exception) as e:
        print(f"Could not retrieve data for strike {target_strike} and expiration {exp}: {e}")
        continue


# === Grafico del prezzo delle Put su diverse scadenze con etichette ===
if put_data_summary:
    exps = list(put_data_summary.keys())
    prices = [data['lastPrice'] for data in put_data_summary.values()]
    implied_vols = [data['impliedVolatility'] for data in put_data_summary.values()]

    plt.figure(figsize=(15, 7))
    plt.plot(exps, prices, marker='o', linestyle='-')

    # Add value labels (price and implied volatility) to each point
    for i, exp in enumerate(exps):
        plt.annotate(f'Price: {prices[i]:.2f}\nIV: {implied_vols[i]:.2f}', (exp, prices[i]), textcoords="offset points", xytext=(0,15), ha='center')


    plt.xlabel("Data di Scadenza")
    plt.ylabel("Prezzo Ultimo Scambio")
    plt.title(f"Prezzo Ultimo Scambio Put Strike {target_strike} su Scadenze Terzo Venerdì del Mese - {ticker_symbol}")
    plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
    plt.tight_layout() # Adjust layout to prevent labels overlapping
    plt.grid(True)
    plt.show()
else:
    print(f"Nessun dato disponibile per le opzioni Put con strike {target_strike} su scadenze del terzo venerdì del mese.")